In [5]:
import torch
from torch import nn
from torch import optim
import gym

In [8]:
env = gym.make('CartPole-v1')
n_actions = env.action_space.n
n_observations = env.observation_space.shape[0]
hidden_size = 150

In [14]:
model = nn.Sequential(
    nn.Linear(n_observations, hidden_size),
    nn.LeakyReLU(),
    nn.Linear(hidden_size, n_actions),
    nn.Softmax(dim=-1)
)

In [15]:
LEARNING_RATE = 0.0009
optimizier = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [16]:
state, _ = env.reset()

In [17]:
state

array([-0.0324774 ,  0.02634265,  0.0240226 , -0.02437359], dtype=float32)

In [18]:
pred = model(torch.from_numpy(state))

In [20]:
def select_random_action(preds):
    pass

In [19]:
action = select_random_action(pred)

tensor([0.4968, 0.5032], grad_fn=<SoftmaxBackward0>)

In [ ]:
new_observation, reward, done, truncated, info = env.step(action)

### Discounted Reward

In [35]:
def discount_rewards(rewards, discount_factor):
    n_rewards = len(rewards)
    timesteps = torch.arange(0, n_rewards)
    
    # calculate the discount for each time step
    discount = torch.pow(discount_factor, timesteps)
    discounted_rewards = discount * rewards
    
    # normalize
    discounted_rewards /= discounted_rewards.max()

    return discounted_rewards

In [36]:
rewards = torch.tensor([1, 2, 3])

In [37]:
discount_rewards(rewards, discount_factor=0.99)

tensor([0.3401, 0.6734, 1.0000])

### Loss Function

$-\gamma_t * G_t * \log \pi_s(a \mid \theta)$

$G_t$: is called the sum of all rewards until time $t$
- $G_t=r_t+r_{t+1} \ldots+r_{T-1}+r_T$

**Example**:

In [38]:
def loss_func(preds, discounted_rewards):
    return -1 * torch.sum(discounted_rewards * torch.log(preds))

### Training Loop

In [40]:
MAX_DUR = 200
MAX_EPISODES = 500

discount_factor = 0.99
score = []

In [ ]:
for episode in range(MAX_EPISODES):
    state, _ = env.reset()
    done= False
    transitions = []
    
    for t in range(MAX_DUR):
        action_probs = model(torch.from_numpy(state)).float()
        action = 